In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *
import qgrid

## Download

In [ ]:
# https://github.com/Crypto-AI/Poloniexport
# https://market.mashape.com/BraveNewCoin/digital-currency-ex-rates#mwa-historic-exchange-rates
# https://bravenewcoin.com/api/digital-currency-exchange-rates/
# http://docs.python-requests.org/en/master/user/quickstart/ 

# Coins
# BTC start = 1396328100 - datetime.datetime(2014, 3, 31, 21, 55)
# ETH start = 1439011500 - datetime.datetime(2015, 8, 7, 22, 25)
# LTC start = 1396344000 - datetime.datetime(2014, 4, 1, 2, 20)
# XRP start = 1397010900 - datetime.datetime(2014, 4, 8, 19, 35)

# Meta
# columns = ['timestamp', 'index', 'volume', 'index_usd', 'volume_usd']
# Returns 1000 rows? Multiple days worth of data...
# 230 requests for 2 years of 5-min data?
# Looks like they fill in missing data with previous known price
# They have 0 volume days

In [ ]:
def write_to_df(data, fpath, columns):
    if os.path.exists(fpath):
        df = pd.read_csv(fpath)
        data = pd.DataFrame(data, columns=columns)
        df.set_index(columns[0])
        df = pd.concat([df, data]).drop_duplicates().reset_index(drop=True)
    else:
        df = pd.DataFrame(data, columns=columns)
        df.set_index(columns[0])
    df.to_csv(fpath, index=False)
    return df

def get_bnc_data(coin, fiat, start_time, end_time):
    url = cfg.BNC_ENDPOINT + '/mwa-historic'
    params = {
        'coin': coin,
        'market': fiat,
        'from': round(start_time.timestamp()),
        'to': round(end_time.timestamp())
    }
    headers = {
        "X-Mashape-Key": cfg.BNC_API_KEY,
        "Accept": "application/json"
    }
    r = requests.get(url, headers=headers, params=params)
    data = r.json()['data']
    data = np.array(data).astype(float)
    return data

def get_all_data(coin, fiat, start_utc, end_utc, timesteps_per_request, timestep_sec, outfpath):
    start_time = datetime.datetime.strptime(
        start_utc, '%Y-%m-%dT%H:%M:%SZ')#.astimezone(datetime.timezone.utc)
    end_time = datetime.datetime.strptime(
        end_utc, '%Y-%m-%dT%H:%M:%SZ')#.astimezone(datetime.timezone.utc)
    cur_time = start_time
    timerange_delta = datetime.timedelta(
        seconds=timesteps_per_request*timestep_sec)
    timestep_delta = datetime.timedelta(seconds=timestep_sec)
    n_records = 0
    retry = 0
    while cur_time < end_time and retry < 10:
        try:
            data = get_bnc_data(coin, fiat, cur_time, cur_time+timerange_delta)
            last_time = datetime.datetime.fromtimestamp(np.max(data[:,0]))
            if last_time < cur_time:
                break
            print("Records", n_records, "Start:", cur_time, "End:", last_time)
            cur_time = last_time + timestep_delta
            df = write_to_df(data, outfpath, BNC_PRICE_COLUMNS)
            n_records += len(data)
            retry = 0
        except Exception as e:
            retry += 1
            print("Error! Retrying!", e)
            traceback.print_exc()
        finally:
            time.sleep(1)
    return df

In [ ]:
BNC_PRICE_COLUMNS = ['timestamp', 'price_coin', 'volume_coin', 'price_fiat', 'volume_fiat']
BNC_EXCHANGE_RATE_ENDPOINT = 'https://bravenewcoin-mwa-historic-v1.p.mashape.com/mwa-historic'
COIN = coins.BTC
FIAT = coins.USD
START_UTC = '2015-08-08T00:00:00Z'
END_UTC = '2017-12-31T00:00:00Z'
TIMESTEP_INTERVAL = 300 # 5 minutes
TIMESTEPS_PER_REQUEST = 1000 
EXCHANGE = c.BNC
PRICE_FNAME = '{:s}_{:s}_{:d}.csv'.format(EXCHANGE, COIN+'-'+FIAT, TIMESTEP_INTERVAL)
PRICE_FPATH = os.path.join(cfg.DATA_DIR, PRICE_FNAME)

In [ ]:
get_bnc_data(coins.BTC, coins.USD, 
             datetime.datetime.fromtimestamp(1396328100), 
             datetime.datetime.fromtimestamp(1396329100))

In [ ]:
df = get_all_data(COIN, FIAT, START_UTC, END_UTC, TIMESTEPS_PER_REQUEST, TIMESTEP_INTERVAL, PRICE_FPATH)

In [ ]:
# BTC
PRICE_FNAME = '{:s}_{:s}_{:d}.csv'.format(EXCHANGE, coins.BTC+'-'+FIAT, TIMESTEP_INTERVAL)
PRICE_FPATH = os.path.join(cfg.DATA_DIR, PRICE_FNAME)
df = get_all_data(coins.BTC, FIAT, START_UTC, END_UTC, TIMESTEPS_PER_REQUEST, TIMESTEP_INTERVAL, PRICE_FPATH)

In [ ]:
# ETH
PRICE_FNAME = '{:s}_{:s}_{:d}.csv'.format(EXCHANGE, coins.ETH+'-'+FIAT, TIMESTEP_INTERVAL)
PRICE_FPATH = os.path.join(cfg.DATA_DIR, PRICE_FNAME)
df = get_all_data(coins.ETH, FIAT, START_UTC, END_UTC, TIMESTEPS_PER_REQUEST, TIMESTEP_INTERVAL, PRICE_FPATH)

In [ ]:
# LTC
PRICE_FNAME = '{:s}_{:s}_{:d}.csv'.format(EXCHANGE, coins.LTC+'-'+FIAT, TIMESTEP_INTERVAL)
PRICE_FPATH = os.path.join(cfg.DATA_DIR, PRICE_FNAME)
df = get_all_data(coins.LTC, FIAT, START_UTC, END_UTC, TIMESTEPS_PER_REQUEST, TIMESTEP_INTERVAL, PRICE_FPATH)

In [ ]:
# XRP
PRICE_FNAME = '{:s}_{:s}_{:d}.csv'.format(EXCHANGE, coins.XRP+'-'+FIAT, TIMESTEP_INTERVAL)
PRICE_FPATH = os.path.join(cfg.DATA_DIR, PRICE_FNAME)
df = get_all_data(coins.XRP, FIAT, START_UTC, END_UTC, TIMESTEPS_PER_REQUEST, TIMESTEP_INTERVAL, PRICE_FPATH)

## Load CSV

In [ ]:
# https://github.com/bfortuner/computer-vision/blob/master/applied/libraries/PandasQuickstart.ipynb

def get_price_fpath(coin, fiat, exchange, timestep):
    fname = '{:s}_{:s}_{:d}.csv'.format(exchange, coin+'-'+fiat, timestep)
    return os.path.join(cfg.DATA_DIR, fname)

def insert_fiat_title(columns, fiat):
    for i in range(len(columns)):
        columns[i].replace('fiat', fiat)
    return columns

def format_bnc_exchange_rate_data(df):
    df = df.rename(index=str, columns={"price_fiat": "price", "volume_fiat": "volume"})
    df = df.drop(labels=['price_coin', 'volume_coin'], axis=1)
    for col in df.columns:
        df[col] = np.round(df[col].values, 5)
    df['timestamp'] = df['timestamp'].values.astype(int)
    df['volume'] = df['volume'].values.astype(int)
    df.sort_values(by='timestamp', ascending=True, inplace=True)
    df = df.set_index('timestamp')
    return df

def load_bnc_exchange_rate_data(fpath):
    df = pd.read_csv(fpath)
    df = format_bnc_exchange_rate_data(df)
    return df

def load_price_df(coin, fiat, exchange, timestep, loader):
    fpath = get_price_fpath(coin, fiat, exchange, timestep)
    df = loader(fpath)
    return df

def join_price_dfs(dfs, coins):
    combo = None
    for i in range(len(dfs)):
        dfs[i].columns = [c + '_' + coins[i] for c in dfs[i].columns]
        if combo is None:
            combo = dfs[i].copy()
        else:
            combo = combo.join(dfs[i].copy(), sort=True)
    epochsec = [datetime.datetime.fromtimestamp(t) for t in combo.index.values]
    combo = combo.assign(utc = epochsec)
    combo.sort_values(by='utc', ascending=True, inplace=True)
    return combo

def load_prices(coins, fiat, exchange, timestep_sec, loader):
    dfs = []
    for coin in coins:
        df = load_price_df(coin, fiat, exchange, timestep_sec, loader)
        dfs.append(df)
    combined = join_price_dfs(dfs, coins)
    return combined

In [ ]:
btc_df = load_price_df(coins.BTC, coins.USD, c.BNC, 300, load_bnc_exchange_rate_data)
eth_df = load_price_df(coins.ETH, coins.USD, c.BNC, 300, load_bnc_exchange_rate_data)
ltc_df = load_price_df(coins.LTC, coins.USD, c.BNC, 300, load_bnc_exchange_rate_data)

In [ ]:
price_df = load_prices(coins=[coins.BTC, coins.ETH, coins.LTC], fiat=coins.USD, exchange=c.BNC, 
                       timestep_sec=300, loader=load_bnc_exchange_rate_data)
price_df.head()

In [ ]:
# Null Handling 
price_df.isnull().sum()                            #Show # of null values in each Column

# df3[df3.Age.isnull()][['Name','Age']]         #Show rows where Age is null
# df3.isnull()                                  #Return True/False if cell is null
# df3.fillna(value={"Cabin":"DEFAULT_CABIN"})   #Set null values in column to default value 
# df3.dropna(how='any')                         #Drop rows with missing data in ANY cell
# df3.fillna(value="DEFAULT").head(2)           #Fill cells missing data w 5

In [ ]:
# Check for missing timesteps
def check_missing_timesteps(df, timestep):
    df = df.sort_values(by='utc')
    start_time = df.iloc[0]['utc']
    end_time = df.iloc[-1]['utc']
    print("Start", start_time)
    print("End", end_time)
    last_time = start_time
    n_missing = 0
    for idx,row in df[1:].iterrows():
        cur_time = row['utc']
        if cur_time != last_time + datetime.timedelta(seconds=timestep):
            print("Expected:", last_time + datetime.timedelta(seconds=timestep), 
                  "| Time:", cur_time)
            n_missing += (cur_time - last_time).seconds//timestep
        last_time = cur_time
    return n_missing

check_missing_timesteps(price_df, 300)

## Plot

In [ ]:
# https://github.com/quantopian/qgrid
# https://hub.mybinder.org/user/quantopian-qgrid-notebooks-bu5joi0d/notebooks/index.ipynb
# https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html

qgrid_widget = qgrid.QgridWidget(df=price_df, show_toolbar=False)
qgrid_widget
#qgrid_widget.get_changed_df()

In [ ]:
def plot_prices(time, close, title="Price"):
    fig, ax = plt.subplots()
    ax.plot(time, close)

    years = mdates.YearLocator()   # every year
    months = mdates.MonthLocator()  # every month
    yearsFmt = mdates.DateFormatter('%Y')
    monthsFmt = mdates.DateFormatter('%m')
    ax.xaxis.set_major_locator(years)
    ax.xaxis.set_major_formatter(yearsFmt)
    ax.xaxis.set_minor_locator(months)
    ax.xaxis.set_minor_formatter(monthsFmt)

    # datemin = datetime.date(r.date.min().year, 1, 1)
    # datemax = datetime.date(r.date.max().year + 1, 1, 1)
    # ax.set_xlim(datemin, datemax)

    # # format the coords message box
    # def price(x):
    #     return '$%1.2f' % x
    # ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')
    # ax.format_ydata = price
    ax.grid(True)

    # rotates and right aligns the x labels, and moves the bottom of the
    # axes up to make room for them
    fig.autofmt_xdate(rotation=30)
    fig.set_size_inches(12,6)
    plt.title(title)
    plt.show()

def plot_range(df, start, end, y_name):
    df = df[ (df['utc'] >= start) & (df['utc'] < end) ]
    vals = df[['utc', y_name]].values
    plot_prices(vals[:,0], vals[:,1], title=y_name)

In [ ]:
vals = price_df[['utc','price_BTC']].values
plot_prices(vals[:,0], vals[:,1], 'BTC-USD')
vals = price_df[['utc','price_ETH']].values
plot_prices(vals[:,0], vals[:,1], 'ETH-USD')
vals = price_df[['utc','price_LTC']].values
plot_prices(vals[:,0], vals[:,1], 'LTC-USD')

start = datetime.datetime(2017, 4, 15, 12, 0)
end = datetime.datetime(2017, 8, 15, 16, 10)
plot_range(price_df, start, end, 'price_BTC')

### Bin

In [ ]:
def get_binned_prices(df, bin_size='30T'):
    # https://stackoverflow.com/questions/17001389/pandas-resample-documentation
    # http://benalexkeen.com/resampling-time-series-data-with-pandas/
    # price_df.resample(rule='30T').ohlc() - (open, high, low, close)
    
    df = df.set_index('utc')
    columns = df.columns
    binned = pd.DataFrame()
    for col in columns:
        if 'price' in col:
            binned['open_' + col] = df[col].resample(rule=bin_size).first().round(5)
            binned['close_' + col] = df[col].resample(rule=bin_size).last().round(5)
            binned['high_' + col] = df[col].resample(rule=bin_size).max().round(5)
            binned['low_' + col] = df[col].resample(rule=bin_size).min().round(5)
        elif 'volume' in col:
            binned[col] = df[col].resample(rule=bin_size).sum().round(0)
        else:
            binned[col] = df[col]
    binned['timestamp'] = [round(t.timestamp()) for t in binned.index]
    return binned

In [ ]:
currency_df = load_prices(coins=[coins.BTC, coins.ETH, coins.LTC], fiat=coins.USD, exchange=c.BNC, 
                       timestep_sec=300, loader=load_bnc_exchange_rate_data)

In [ ]:
currency_30min_df = get_binned_prices(currency_df, bin_size='30T')
currency_30min_df.head()

In [ ]:
qgrid_widget = qgrid.QgridWidget(df=currency_30min_df, show_toolbar=False)
qgrid_widget

## Analyze

In [ ]:
columns = [c for c in currency_30min_df.columns if 'close_price' in c]
price_df = currency_30min_df[columns]
price_df.columns = ['BTC', 'ETH', 'LTC']
price_df.head()

In [ ]:
# Normalize 
price_df_norm = price_df.divide(price_df.iloc[0])
price_df_norm.head()

In [ ]:
price_df_norm.plot()

In [ ]:
# Percent change (from ?)
# There are some outliers that will mess up a model. How to fix....
# for example, BTC on 2016-09-26 06:00:00
price_df_pct = price_df.pct_change()
price_df_pct = price_df_pct.replace([1.0, -1.0], np.nan)
price_df_pct = price_df_pct.replace([np.inf, -np.inf], np.nan)
price_df_pct = price_df_pct.dropna()
price_df_pct.head()

In [ ]:
# Correlation
# https://www.investopedia.com/terms/n/negative-correlation.asp
# https://medium.com/@eliquinox/cryptocurrency-data-analysis-part-ii-downloading-manipulating-and-analysing-bulk-data-e234a43e6259

corr = price_df_pct.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

# Print correlations
corr['BTC']['ETH'], corr['BTC']['LTC']

In [ ]:
plt.scatter(price_df_pct['BTC'], price_df_pct['LTC'])
plt.xlabel('BTC % Return')
plt.ylabel('LTC % Return')

In [ ]:
# Linear Regression
# https://www.datarobot.com/blog/ordinary-least-squares-in-python/
# http://www.statsmodels.org/dev/examples/notebooks/generated/ols.html

import statsmodels.api as sm

# Dependent, Indenpendent
model = sm.OLS(price_df_pct['ETH'],
               price_df_pct['LTC']).fit()
model.summary()

# R squared - how much of LTC change can be explained by BTC changes?

In [ ]:
# multiply each value of BTC pct change by the slope coefficient
line = [model.params[0]*i for i in price_df_pct['BTC'].values]
plt.plot(price_df_pct['BTC'], line, c = 'r')
plt.scatter(price_df_pct['BTC'], price_df_pct['LTC'])
plt.xlabel('BTC % Return')
plt.ylabel('LTC % Return')

## Moving Average Crossover

In [ ]:
# https://en.wikipedia.org/wiki/Moving_average#Simple_moving_average
"""
Interestingly it looks like prices quoted in Bitcoin are more predictive ..
"""

coin = coins.LTC
fiat = coins.USD

df = load_prices(coins=[coin], fiat=fiat, exchange=c.BNC, 
                 timestep_sec=300, loader=load_bnc_exchange_rate_data)
df = df.set_index('utc')
df = df.rename(columns={'price_'+coin:'close'})
df = df.drop(['volume_'+coin], axis=1)

In [ ]:
df.head()

In [ ]:
# Add moving averages
df['SMA_1000'] = df['close'].rolling(1000).mean() # 16 hours
df['SMA_5000'] = df['close'].rolling(5000).mean() # 4 days
df[['close','SMA_1000','SMA_5000']][len(df)-15000:].plot(figsize = (16,10))
df = df.dropna()
df.head()

In [ ]:
"""
Strategy

BUY if Leading SMA is above Lagging SMA by some threshold.
SELL if Leading SMA is below Lagging SMA by some threshold.

Threshold = 2.5% of current price (filter out weak signals)
"""

def test_sma(df, lead, lag, pc_thresh = 0.025):
    ma_df = df.copy()
    ma_df['lead'] = ma_df['close'].rolling(lead).mean()
    ma_df['lag'] = ma_df['close'].rolling(lag).mean()
    ma_df.dropna(inplace = True)
    ma_df['lead-lag'] = ma_df['lead'] - ma_df['lag']
    ma_df['pc_diff'] = ma_df['lead-lag'] / ma_df['close']
    ma_df['regime'] = np.where(ma_df['pc_diff'] > pc_thresh, 1, 0)
    ma_df['regime'] = np.where(ma_df['pc_diff'] < -pc_thresh, -1, ma_df['regime'])
    ma_df['Market'] = np.log(ma_df['close'] / ma_df['close'].shift(1))
    ma_df['Strategy'] = ma_df['regime'].shift(1) * ma_df['Market']
    ma_df[['Market','Strategy']] = ma_df[['Market','Strategy']].cumsum().apply(np.exp)
    return ma_df

In [ ]:
sma_df = test_sma(df, 100, 4100).dropna()
sma_df.head()

In [ ]:
sma_df['Strategy'][-1]

In [ ]:
# Buy / Short / Neutral Action
sma_df['regime'].plot(figsize=(16,5))

In [ ]:
# Compare to buy and hold
sma_df[['Market','Strategy']].iloc[-1]

In [ ]:
sma_df[['Market','Strategy']][len(sma_df)-100000:].plot(figsize = (16,10))

In [ ]:
# Search for Optimal Lead/Lag Periods
leads = np.arange(100, 4100, 100)
lags = np.arange(4100, 8100, 100)
lead_lags = [[lead, lag] for lead in leads for lag in lags]
pnls = pd.DataFrame(index=lags,  columns=leads)
print(len(lead_lags))

In [ ]:
for lead, lag in lead_lags:
    results = test_sma(df, lead, lag).dropna()
    pnls[lead][lag] = results['Strategy'][-1]
    print(lead, lag, pnls[lead][lag])

In [ ]:
pnls.head()

In [ ]:
PNLs = pnls[pnls.columns].astype(float)
plt.subplots(figsize = (14,10))
sns.heatmap(PNLs, cmap='PiYG')

In [ ]:
# Get max value for each lead lookback
"""
Lead of 900 periods produces maximum PnL. 
By referencing back the the matrix heatmap, we can find the corresponding lag period.

Optimal looks like (BTC):
    lead = 1200 - 1500
    lag = 5400 - 5800

lets say (1350, 5800)
ratio: 5:1
"""
PNLs.max()
print("optimal", PNLs[1300][5800])

## Assumptions

We ignore important factors:

* Commissions (25 bps)
* Shorting (usually can't do this)
* Slippage (we assume we will always get the order price we request)
* Market Impact (the market price may change when we start placing orders)
* Overfitting (We brute force searched for optimal lead/lag ratio - this won't generalize)
* Exchange risk (Exchanges get hacked)